In [57]:
import my_sql_module as msm

PGHOST = '...'
PGDATABASE = '...'
PGUSER = '...'
PGPASSWORD = '..'

In [58]:
# Una conexión para todo el programa
conn = msm.create_server_connection(PGHOST, PGDATABASE, PGUSER, PGPASSWORD)

Database conn successful


In [59]:
# Total de pedidos
def total_de_pedidos_unicos():

    cur = None

    data = None

    SQL = 'SELECT COUNT(DiSTINCT checkNumber) FROM payments'
    print('SQL:', SQL)

    try:
        cur = conn.cursor()
        cur.execute(SQL)
        data = cur.fetchall()
    except Exception as err:
        print(f"Error: '{err}'")
        if conn: conn.rollback()
    finally:
        if cur:
            cur.close()

    return data[0][0]


resultado = total_de_pedidos_unicos()

print(resultado)

SQL: SELECT COUNT(DiSTINCT checkNumber) FROM payments
259


In [60]:
# La cantidad media de items que se pidió.

def obtener_items_x_venta_ddbb():
    cur = None
    data = None

    SQL = 'SELECT  DISTINCT orderNumber, orderlinenumber, quantityOrdered  FROM sales GROUP BY orderNumber,orderlinenumber ORDER BY orderNumber'
    print('SQL:', SQL)

    try:
        cur = conn.cursor()
        cur.execute(SQL)
        data = cur.fetchall()
    except Exception as err:
        print(f"Error: '{err}'")
        if conn:
            conn.rollback()
    finally:
        if cur:
            cur.close()

    return data


def cantidad_media_de_items():
    data = obtener_items_x_venta_ddbb()

    curr_order = None
    qitems_x_venta = {}
    suma_items_pedidos = 0

    for (ordn, lordn, q) in data:
        if ordn != curr_order:
            curr_order = ordn
            qitems_x_venta[ordn] = 0

        qitems_x_venta[ordn] += q
        suma_items_pedidos += q

    media = round(suma_items_pedidos/len(qitems_x_venta), 2)

    return media, qitems_x_venta


media, iventas = cantidad_media_de_items()
print('media:',media)
print('ventas:', iventas)

SQL: SELECT  DISTINCT orderNumber, orderlinenumber, quantityOrdered  FROM sales GROUP BY orderNumber,orderlinenumber ORDER BY orderNumber
media: 324.49
ventas: {10100: 151, 10101: 142, 10102: 80, 10103: 541, 10104: 443, 10105: 545, 10106: 675, 10107: 229, 10109: 212, 10110: 570, 10111: 217, 10112: 52, 10113: 143, 10114: 351, 10115: 210, 10116: 27, 10118: 36, 10119: 442, 10120: 525, 10121: 185, 10122: 545, 10123: 156, 10124: 448, 10125: 66, 10126: 617, 10127: 540, 10128: 157, 10129: 349, 10130: 73, 10131: 244, 10132: 36, 10133: 286, 10134: 227, 10135: 607, 10136: 102, 10137: 138, 10138: 420, 10139: 266, 10140: 385, 10141: 290, 10142: 577, 10143: 524, 10144: 20, 10145: 563, 10146: 76, 10147: 341, 10148: 430, 10149: 367, 10151: 342, 10152: 116, 10153: 458, 10154: 67, 10155: 454, 10156: 68, 10157: 216, 10158: 22, 10159: 585, 10160: 231, 10161: 358, 10162: 391, 10163: 225, 10164: 288, 10166: 98, 10167: 550, 10168: 642, 10169: 444, 10170: 142, 10171: 145, 10172: 263, 10173: 480, 10174: 220, 

In [61]:
# Aquellos clientes que tienen un crédito superior a 1000. 

import statistics as st

def clientes_credito_mayor_a(valor, esMayor):
    cur = None
    data = None

    simbolo = '>'
    if not esMayor: simbolo = '<='

    SQL = 'SELECT  c.* from customers c WHERE c.creditlimit {0} {1}'.format(
        simbolo, valor)
    print('SQL:', SQL)

    try:
        cur = conn.cursor()
        cur.execute(SQL)
        data = cur.fetchall()
    except Exception as err:
        print(f"Error: '{err}'")
        if conn:
            conn.rollback()
    finally:
        if cur:
            cur.close()

    return data


def getClientesMediaVarianza(valor, esMayor=True):
    clientes = clientes_credito_mayor_a(valor, esMayor)

    lcredits = list(map(lambda aRow: float(aRow[-1]), clientes))

    return clientes, st.mean(lcredits), st.variance(lcredits)


clientes, media, varianza = getClientesMediaVarianza(1000)
print('clientes:', clientes)
print('media:', media)
print('varianza:', varianza)

SQL: SELECT  c.* from customers c WHERE c.creditlimit > 1000
clientes: [(103, 'Atelier graphique', 'Carine  Schmitt', '40.32.2555', '54, rue Royale', 'Nantes', None, '44000', 'France', 1370, Decimal('21000.00')), (114, 'Australian Collectors, Co.', 'Peter Ferguson', '03 9520 4555', '636 St Kilda Road, Level 3', 'Melbourne', 'Victoria', '3004', 'Australia', 1611, Decimal('117300.00')), (119, 'La Rochelle Gifts', 'Janine  Labrune', '40.67.8555', '67, rue des Cinquante Otages', 'Nantes', None, '44000', 'France', 1370, Decimal('118200.00')), (121, 'Baane Mini Imports', 'Jonas  Bergulfsen', '07-98 9555', 'Erling Skakkes gate 78', 'Stavern', None, '4110', 'Norway', 1504, Decimal('81700.00')), (124, 'Mini Gifts Distributors Ltd.', 'Susan Nelson', '4155551450', '5677 Strong St.', 'San Rafael', 'CA', '97562', 'USA', 1165, Decimal('210500.00')), (128, 'Blauer See Auto, Co.', 'Roland Keitel', '+49 69 66 90 2555', 'Lyonerstr. 34', 'Frankfurt', None, '60528', 'Germany', 1504, Decimal('59700.00')), 

In [62]:
# Aquellos clientes que tienen un crédito inferior o igual a 1000. 

clientes, media, varianza = getClientesMediaVarianza(1000, esMayor=False)
print('clientes:', clientes)
print('media:', media)
print('varianza:', varianza)

SQL: SELECT  c.* from customers c WHERE c.creditlimit <= 1000
clientes: [(125, 'Havel & Zbyszek Co', 'Zbyszek  Piestrzeniewicz', '(26) 642-7555', 'ul. Filtrowa 68', 'Warszawa', None, '01-012', 'Poland', None, Decimal('0.00')), (168, 'American Souvenirs Inc', 'Keith Franco', '2035557845', '149 Spinnaker Dr., Suite 101', 'New Haven', 'CT', '97823', 'USA', 1286, Decimal('0.00')), (169, 'Porto Imports Co.', 'Isabel  de Castro', '(1) 356-5555', 'Estrada da saúde n. 58', 'Lisboa', None, '1756', 'Portugal', None, Decimal('0.00')), (206, 'Asian Shopping Network, Co', 'Brydey Walker', '+612 9411 1555', 'Suntec Tower Three, 8 Temasek', 'Singapore', None, '038988', 'Singapore', None, Decimal('0.00')), (223, 'Natürlich Autos', 'Horst  Kloss', '0372-555188', 'Taucherstraße 10', 'Cunewalde', None, '01307', 'Germany', None, Decimal('0.00')), (237, 'ANG Resellers', 'Alejandra  Camino', '(91) 745 6555', 'Gran Vía, 1', 'Madrid', None, '28001', 'Spain', None, Decimal('0.00')), (247, 'Messner Shopping Net

In [63]:
# Cerramos conexión global
if conn: conn.close()